## Performance Analysis of Optimization Algorithms for Neural Networks
This notebook contains implementations of the Adam optimizer and the key algorithms prior to it. These include Gradient Descent, Momentum, Adagrad, and RMSprop. Further, existing variations of Adam, such as Adamax, AdaBelief & Nadam are implemented.<br><br>
The goal is to minimize the **loss function of a neural network** with the following architecture, where each number represents the number of neurons in a layer: [4, 5, 5, 12, 1, 22, 2].

Supporting materials:
<br>
_[Andrej Karpathy's repository](https://github.com/karpathy/micrograd/tree/master/micrograd) (micrograd)_
<br>
_[Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980) (original paper on Adam)_ 
<br>
_[A journey into Optimization algorithms for Deep Neural Networks](https://theaisummer.com/optimization/) (a nice artice on the math behind neural network optimizers)_


In [2]:
import numpy as np

### NN with Gradient Descent

In [ ]:
class NNGD:
    def __init__(self, layer_sizes, learning_rate = 0.1):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate

        self.weights = []
        self.biases = []

        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            ##########################################
            # GRADIENT DESCENT

            # (w1 = w0 - (learning rate * (dL/dW)))
            self.weights[i] -= self.learning_rate * dW

            # (b1 = b0 - (learning rate * (dL/db)))
            self.biases[i] -= self.learning_rate * db 
            
            ##########################################

    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-8
        while loss >= 1e-8:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNGD((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 73258
Final loss: 1e-08


### NN with Momentum

**Notes:**<br>
1. There are 2 new parameters here; Velocity and Friction. Velocity is calculated as the running mean of gradients.<br>
2. This forces the gradients to keep moving at each time step. Friction is just a constant that aims to decay this effect.<br>
3. Basically, the function value at the previous time step is decayed slightly to give it less importance while giving full importance to the new function value (learning_rate * slope). This is saved into a variable. Next, this new variable will be decayed. Iteratively, a moving average (or smoothing) of previous function values are stored. 

In [ ]:
class NNMomentum:
    def __init__(self, layer_sizes, learning_rate = 0.1, friction = 1):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.friction = friction

        self.weights = []
        self.biases = []

        self.velocity_w = []
        self.velocity_b = []

        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

            self.velocity_w.append(np.zeros_like(w))
            self.velocity_b.append(np.zeros_like(b))

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            #####################################################################################
            # MOMENTUM (averaging is partially caused due to the friction element. (check notes for in-depth understanding))

            # velocity_w_1 = (friction * velocity_w_0) + (l_r * dL/dW)
            self.velocity_w[i] = (self.friction * self.velocity_w[i]) + (self.learning_rate * dW)
            # velocity_b_1 = (friction * velocity_b_0) + (l_r * dL/db)
            self.velocity_b[i] = (self.friction * self.velocity_b[i]) + (self.learning_rate * db)

            self.weights[i] -= self.velocity_w[i]
            self.biases[i] -= self.velocity_b[i]
            
            #####################################################################################
    
    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-8
        while loss >= 1e-8:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNMomentum((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 415
Final loss: 1e-08


### NN with Adagrad

**Notes:**<br>
1. Learning rate shrinks at each time step. 
2. Initial learning rate is large as the initial value at the denominator is small.
3. As the gradients are accumulated, the denominator grows which leads to learning rate reduction.
4. **Drawback**: If the network is very large, over accumulation of gradients will make the denominator very large leading to extremely slow updates.

In [ ]:
class NNAdagrad:
    def __init__(self, layer_sizes, learning_rate = 0.1, epsilon = 1e-8):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.epsilon = epsilon

        self.weights = []
        self.biases = []

        self.sq_grads_w = []
        self.sq_grads_b = []

        self.stuff_w = []
        self.stuff_b = []


        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

            self.sq_grads_w.append(np.zeros_like(w))
            self.sq_grads_b.append(np.zeros_like(b))

            self.stuff_w.append(np.zeros_like(w))
            self.stuff_b.append(np.zeros_like(b))

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            ###########################################################################################
            # ADAGRAD
            
            # sum of squared gradients
            self.sq_grads_w[i] += np.power(dW, 2)
            self.sq_grads_b[i] += np.power(db, 2)
            
            # updation
            self.weights[i] -= self.learning_rate * (dW / (np.sqrt(self.sq_grads_w[i]) + self.epsilon))
            self.biases[i] -= self.learning_rate * (db / (np.sqrt(self.sq_grads_b[i]) + self.epsilon))
            
            ###########################################################################################

    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-8
        while loss >= 1e-8:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNAdagrad((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 13697
Final loss: 1e-08


### NN with RMSProp

**Notes:**<br>
1. The issue where the derivatives kept gowring at every step without any limit due to the squaring of the gradient is eliminated her.
2. This is done by multiplying `(1 - beta)` to `dw_squared`.

__Testing:__<br>
1. Why not take mod of the gradients instead of square?

In [ ]:
class NNRMSProp:
    def __init__(self, layer_sizes, learning_rate = 0.1, decay = 0.3, epsilon = 1e-8):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.decay = decay
        self.epsilon = epsilon

        self.weights = []
        self.biases = []

        self.sq_grads_w = []
        self.sq_grads_b = []

        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

            self.sq_grads_w.append(np.zeros_like(w))
            self.sq_grads_b.append(np.zeros_like(b))

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            ###########################################################################################
            # RMSProp

            # decay parameter lowers the effect of the squaring of the derivatives            
            self.sq_grads_w[i] = self.decay * self.sq_grads_w[i] + (1 - self.decay) * np.power(dW, 2)
            self.sq_grads_b[i] = self.decay * self.sq_grads_b[i] + (1 - self.decay) * np.power(db, 2)

            self.weights[i] -= self.learning_rate * (dW / (np.sqrt(self.sq_grads_w[i]) + self.epsilon))
            self.biases[i] -= self.learning_rate * (db / (np.sqrt(self.sq_grads_b[i]) + self.epsilon))

            # ADAGRAD (FOR COMPARISON)
            
            # self.sq_grads_w[i] += np.power(dW, 2)
            # self.sq_grads_b[i] += np.power(db, 2)

            # self.weights[i] -= self.learning_rate * (dW / (np.sqrt(self.sq_grads_w[i]) + self.epsilon))
            # self.biases[i] -= self.learning_rate * (db / (np.sqrt(self.sq_grads_b[i]) + self.epsilon))
            
            #############################################################################################

    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-8
        while loss >= 1e-8:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNRMSProp((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 40
Final loss: 6.7e-09


### NN with Adam

**Notes:**<br>
1. Here Momentum & RMSProp is combined.
2. RMSProp stays the same while the Momentum formula is slighlty tweaked for normalization.

In [ ]:
class NNAdam:
    def __init__(self, layer_sizes, learning_rate = 0.1, delta_one = 0.8, delta_two = 0.5, epsilon = 1e-8):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.delta_one = delta_one
        self.delta_two = delta_two
        self.epsilon = epsilon

        self.weights = []
        self.biases = []

        self.moment_one_w = []
        self.moment_one_b = []
        self.moment_two_w = []
        self.moment_two_b = []

        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

            self.moment_one_w.append(np.zeros_like(w))
            self.moment_one_b.append(np.zeros_like(b))
            self.moment_two_w.append(np.zeros_like(w))
            self.moment_two_b.append(np.zeros_like(b))

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y, epoch):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            ###################################################################################################################
            # ADAM

            # MOMENTUM (FOR COMPARISON)

            # self.velocity_w[i] = (self.friction * self.velocity_w[i]) + (self.learning_rate * dW)
            # self.velocity_b[i] = (self.friction * self.velocity_b[i]) + (self.learning_rate * db)

            # self.weights[i] -= self.velocity_w[i]
            # self.biases[i] -= self.velocity_b[i]

            # this is momentum
            self.moment_one_w[i] = self.delta_one * self.moment_one_w[i] + (1 - self.delta_one) * dW # moment one
            self.moment_one_b[i] = self.delta_one * self.moment_one_b[i] + (1 - self.delta_one) * db # moment one

            # this is rmsprop
            self.moment_two_w[i] = self.delta_two * self.moment_two_w[i] + (1 - self.delta_two) * np.power(dW, 2) # moment two
            self.moment_two_b[i] = self.delta_two * self.moment_two_b[i] + (1 - self.delta_two) * np.power(db, 2) # moment two
 
            # bias correction
            moment_one_w_hat = self.moment_one_w[i] / (1 - np.power(self.delta_one, epoch + 1))
            moment_one_b_hat = self.moment_one_b[i] / (1 - np.power(self.delta_one, epoch + 1))

            moment_two_w_hat = self.moment_two_w[i] / (1 - np.power(self.delta_two, epoch + 1))
            moment_two_b_hat = self.moment_two_b[i] / (1 - np.power(self.delta_two, epoch + 1))

            # parameters update
            self.weights[i] -= self.learning_rate * (moment_one_w_hat / (np.sqrt(moment_two_w_hat) + self.epsilon))
            self.biases[i] -= self.learning_rate * (moment_one_b_hat / (np.sqrt(moment_two_b_hat) + self.epsilon))

            ###################################################################################################################

    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-8
        while loss >= 1e-8:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y, epoch) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNAdam((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 30
Final loss: 1.3e-09


### NN with AdaMax

**Notes:**<br>
1. With Adam, when calculating the second moment estimate, we sort of calculate the L2 norm of the gradients. 
2. With AdaMax, instead of squaring dW, we take the maximum absolute value between the current gradient and the past gradients, which is decayed using the decay factor.

In [ ]:
class NNAdaMax:
    def __init__(self, layer_sizes, learning_rate = 0.1, delta_one = 0.7, delta_two = 0.7, inf_norm_w = 1e-8, inf_norm_b = 1e-8, epsilon = 1e-8):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.delta_one = delta_one
        self.delta_two = delta_two
        self.inf_norm_w = inf_norm_w
        self.inf_norm_b = inf_norm_b
        self.epsilon = epsilon

        self.weights = []
        self.biases = []

        self.moment_one_w = []
        self.moment_one_b = []

        self.inf_norm_w = []
        self.inf_norm_b = []

        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

            self.moment_one_w.append(np.zeros_like(w))
            self.moment_one_b.append(np.zeros_like(b))

            self.inf_norm_w.append(np.zeros_like(w))
            self.inf_norm_b.append(np.zeros_like(b))

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y, epoch):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            ################################################################################################
            # ADAMAX

            # first moment estimate
            self.moment_one_w[i] = self.delta_one * self.moment_one_w[i] + (1 - self.delta_one) * dW
            self.moment_one_b[i] = self.delta_one * self.moment_one_b[i] + (1 - self.delta_one) * db

            # infinity norm (max norm of past gradients instead of squaring)
            self.inf_norm_w[i] = np.maximum(self.delta_two * self.inf_norm_w[i], np.abs(dW))
            self.inf_norm_b[i] = np.maximum(self.delta_two * self.inf_norm_b[i], np.abs(db))

            # bias correction
            moment_one_w_hat = self.moment_one_w[i] / (1 - (np.power(self.delta_one, epoch + 1)))
            moment_one_b_hat = self.moment_one_b[i] / (1 - (np.power(self.delta_one, epoch + 1)))

            # parameters update
            self.weights[i] -= (self.learning_rate * moment_one_w_hat) / (self.inf_norm_w[i] + self.epsilon)
            self.biases[i] -= (self.learning_rate * moment_one_b_hat) / (self.inf_norm_b[i] + self.epsilon)
            
            ################################################################################################

    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-8
        while loss >= 1e-8:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y, epoch) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNAdaMax((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 28
Final loss: 7.7e-09


### NN with AdaBelief

**Notes:**<br>
1. In AdaBelief, we tweak the original Adam algorithm, yet again with the second moment estimate. 
2. We subtract the gradient with alpha from the previous time step.

In [ ]:
class NNAdaBelief:
    def __init__(self, layer_sizes, learning_rate = 0.1, delta_one = 0.4, delta_two = 0.2, epsilon = 1e-8):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.delta_one = delta_one
        self.delta_two = delta_two
        self.epsilon = epsilon

        self.weights = []
        self.biases = []

        self.moment_one_w = []
        self.moment_one_b = []
        self.moment_two_w = []
        self.moment_two_b = []

        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

            self.moment_one_w.append(np.zeros_like(w))
            self.moment_one_b.append(np.zeros_like(b))
            self.moment_two_w.append(np.zeros_like(w))
            self.moment_two_b.append(np.zeros_like(b))

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y, epoch):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            ############################################################################################################################
            # ADABELIEF

            # moments calculation
            self.moment_one_w[i] = self.delta_one * self.moment_one_w[i] + (1 - self.delta_one) * dW
            self.moment_one_b[i] = self.delta_one * self.moment_one_b[i] + (1 - self.delta_one) * db

            self.moment_two_w[i] = self.delta_two * self.moment_two_w[i] + (1 - self.delta_two) * np.power(dW - self.moment_two_w[i], 2)
            self.moment_two_b[i] = self.delta_two * self.moment_two_b[i] + (1 - self.delta_two) * np.power(db - self.moment_two_b[i], 2)

            # bias correction
            moment_one_w_hat = self.moment_one_w[i] / (1 - np.power(self.delta_one, epoch + 1))
            moment_one_b_hat = self.moment_one_b[i] / (1 - np.power(self.delta_one, epoch + 1))

            moment_two_w_hat = self.moment_two_w[i] / (1 - np.power(self.delta_two, epoch + 1))
            moment_two_b_hat = self.moment_two_b[i] / (1 - np.power(self.delta_two, epoch + 1))

            # parameters update
            self.weights[i] -= self.learning_rate * (moment_one_w_hat / (np.sqrt(moment_two_w_hat) + self.epsilon))
            self.biases[i] -= self.learning_rate * (moment_one_b_hat / (np.sqrt(moment_two_b_hat) + self.epsilon))

            ############################################################################################################################

    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-8
        while loss >= 1e-8:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y, epoch) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNAdaBelief((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 23
Final loss: 8.1e-09


### NN with Nadam

**Notes:**<br>
1. Nadam is another modification to Adam. 
2. The only difference is that instead of the classic Momentum formula, we replace it with Nesterov Momentum. 
3. This has proven to work better with very noisy gradients.

In [ ]:
class NNNadam:
    def __init__(self, layer_sizes, learning_rate = 0.1, delta_one = 0.6, delta_two = 0.5, epsilon = 1e-8):
        np.random.seed(0)
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.delta_one = delta_one
        self.delta_two = delta_two
        self.epsilon = epsilon

        self.weights = []
        self.biases = []

        self.moment_one_w = []
        self.moment_one_b = []
        self.moment_two_w = []
        self.moment_two_b = []

        for i in range(len(layer_sizes) - 1):

            # initialize random weights and biases
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)

            self.moment_one_w.append(np.zeros_like(w))
            self.moment_one_b.append(np.zeros_like(b))
            self.moment_two_w.append(np.zeros_like(w))
            self.moment_two_b.append(np.zeros_like(b))

    # activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # activation function gradient (for backpropagation)
    def sigmoid_gradient(self, x):
        sx = self.sigmoid(x)
        return sx * (1 - sx)

    def forward_propagation(self, X):
        self.activations = [X] # activation - input matrix
        self.z_values = [] # z_values - intermediate matrix before activation (sigmoid) is applied

        activation = X
        # perform matrix multiplication (xw + b)
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activation, w) + b
            self.z_values.append(z)
            activation = self.sigmoid(z)
            self.activations.append(activation) 

        return activation # final activated output (sigmoid(xw + b))

    def backpropagation(self, X, y, epoch):
        m = X.shape[0] # training data size
        delta = self.activations[-1] - y # error (y_cap - y)

        for i in range(len(self.weights) - 1, -1, -1): # this loop starts from the last layer, all the way to the first layer
            dW = np.dot(self.activations[i].T, delta) / m # derivative of error w.r.t weights
            db = np.sum(delta, axis=0, keepdims=True) / m # derivative of error w.r.t biases

            if i > 0: # we cannnot include the input however
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_gradient(self.z_values[i - 1]) # error update 

            ########################################################################################################
            # NESTROV-ACCELERATED ADAM (NADAM)

            # moments calculation
            self.moment_one_w[i] = self.delta_one * self.moment_one_w[i] + (1 - self.delta_one) * dW
            self.moment_one_b[i] = self.delta_one * self.moment_one_b[i] + (1 - self.delta_one) * db

            self.moment_two_w[i] = self.delta_two * self.moment_two_w[i] + (1 - self.delta_two) * np.power(dW, 2)
            self.moment_two_b[i] = self.delta_two * self.moment_two_b[i] + (1 - self.delta_two) * np.power(db, 2)

            # bias correction
            moment_one_w_hat = self.moment_one_w[i] / (1 - np.power(self.delta_one, epoch + 1))
            moment_one_b_hat = self.moment_one_b[i] / (1 - np.power(self.delta_one, epoch + 1))

            moment_two_w_hat = self.moment_two_w[i] / (1 - np.power(self.delta_two, epoch + 1))
            moment_two_b_hat = self.moment_two_b[i] / (1 - np.power(self.delta_two, epoch + 1))

            # nesterov lookahead adjustment
            moment_one_w_star = (1 - self.delta_one) * dW + self.delta_one * moment_one_w_hat
            moment_one_b_star = (1 - self.delta_one) * db + self.delta_one * moment_one_b_hat

            # parameters update
            self.weights[i] -= self.learning_rate * (moment_one_w_star / (np.sqrt(moment_two_w_hat) + self.epsilon))
            self.biases[i] -= self.learning_rate * (moment_one_b_star / (np.sqrt(moment_two_b_hat) + self.epsilon))

            ########################################################################################################

    def train(self, X, y):
        losses = []
        epoch = 0
        loss = 1e-3
        while loss >= 1e-3:
            output = self.forward_propagation(X) # perform forward propagation
            loss = np.mean(np.square(output - y)) # calculate mean squared error loss
            losses.append(loss)
            self.backpropagation(X, y, epoch) # perform backpropagation   
            epoch += 1        
        print(f"Total number of epochs: {epoch}\nFinal loss: {loss.round(10)}")

# run neural network with gradient descent
X = np.array([[5, 4, 2, -3],
              [3, 1, -5, 0],
              [9, -9, 3, 9],
              [3, 1, -5, 0]])
y = np.array([[1],
              [0],
              [0],
              [0]])

nn = NNNadam((4, 5, 5, 12, 1, 22, 2))
nn.train(X, y)

Total number of epochs: 18
Final loss: 0.0007338104


#### The reduction in steps to converge from 73258 steps (gradient descent) to 18 steps (nadam) is amazing!